# Image Classification using CNN - streamlined w/Fastai

## Review: easy steps to train a world-class image classifier

1. Enable data augmentation, and precompute=True
1. Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs
1. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer
1. Use Test Time Augmentation (TTA) for test predictions
1. Use `lr_find()` again
1. Train full network with cycle_mult=2 until over-fitting

In [24]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [25]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = "data/dogscats/"

sz=224

arch=resnet34

bs=28

In [26]:
# Uncomment the below if you need to reset your precomputed activations
!rm -rf {PATH}tmp

## 1. Enable data augmentation  && Create Data Object

In [27]:
# The transformation object
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

# The cleaned up object which holds our data
data = ImageClassifierData.from_paths(PATH, tfms=tfms)

# The Pretrained ConvLearner object
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 32/32 [00:05<00:00,  5.90it/s]


## 2. Find highest learning rate where loss is still increasing

In [28]:
lrf=learn.lr_find()

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

 76%|███████▋  | 275/360 [00:05<00:01, 47.49it/s, loss=0.456] 


## 3. Train last layer from pre-computed model

In [29]:
learn.fit(1e-2, 1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       0.06501  0.0276   0.98975]                         



## 4. Train last layer w/Data Aug.

In [31]:
learn.precompute=False

In [32]:
learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       0.0426   0.02729  0.98975]                         
[ 1.       0.04244  0.02747  0.99072]                         



## 5. Unfreeze all layers

In [33]:
learn.unfreeze()

## 6. Set earlier layers to 3x-10x lower learning rate than next higher layer

In [ ]:
lr=np.array([1e-4,1e-3,1e-2]) #differential learning rates

learn.fit(lr, 3, cycle_len=1, cycle_mult=2) #cycle_mult=2 needed for resnet34

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.04892  0.02743  0.98682]                         
[ 1.       0.03891  0.02143  0.98828]                         
[ 2.       0.02655  0.02041  0.99121]                         
[ 3.       0.02709  0.01984  0.99121]                         
[ 4.       0.02545  0.01652  0.99365]                         
[ 5.       0.02179  0.01746  0.99316]                         
 16%|█▋        | 59/360 [00:29<02:32,  1.97it/s, loss=0.0162]

In [ ]:
learn.sched.plot()

In [ ]:
learn.save('224_all')

In [ ]:
learn.load('224_all')

## 7. Use Test Time Augmentation for Training Test Predictions

In [ ]:
# Check predictions at this point
log_preds,y = learn.TTA()
accuracy(log_preds,y)

## 8. Find the optimal learning rate again (lr_find)

## 9. Train full network with cycle_mult=2 until over-fitting